In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [3]:
seed=7
np.random.seed(seed)

In [4]:
df=pd.read_excel(r'C:\Users\Lenovo\Downloads\Data.xlsx')
df1=df.values
df

,Current 1,P_L 1,Current 2,P_L 2,Current 3,P_L 3,Current Ideal,P_L Ideal,Time
0,8.858,492.0,5.249,1093.0,1.9690,1640,9.465000e-05,1968,1
1,4.429,246.0,2.625,546.7,0.9844,820,8.514000e-05,984,2
2,4.429,246.0,2.625,546.7,0.9844,820,-9.609000e-11,984,3
3,4.429,246.0,2.625,546.7,0.9844,820,-9.609000e-11,984,4
4,4.429,246.0,2.625,546.7,0.9844,820,-9.609000e-11,984,5
5,7.382,410.0,4.374,911.1,1.6400,1670,5.178000e-06,1640,6
6,14.760,819.9,8.748,1822.0,3.2800,2733,-9.125000e-05,3280,7
7,26.570,1476.0,15.750,3280.0,5.9050,4919,-1.579000e-04,5903,8
8,29.530,1640.0,17.500,3644.0,6.5610,5466,5.177000e-06,6559,9
9,14.760,819.9,8.749,1822.0,3.2810,2733,3.462000e-04,3280,10


In [5]:
po=pd.DataFrame(columns=['current','load','result','Time'])

In [6]:
for i in range(23):
    po=po.append({'current':df.at[i,"Current 1"],'load':df.at[i,"P_L 1"],'result':1,'Time':df.at[i,"Time"]},ignore_index=True)

In [7]:
for i in range(23):
    po=po.append({'current':df.at[i,"Current 2"],'load':df.at[i,"P_L 2"],'result':2,'Time':df.at[i,"Time"]},ignore_index=True)

In [8]:
for i in range(23):
    po=po.append({'current':df.at[i,"Current 3"],'load':df.at[i,"P_L 3"],'result':3,'Time':df.at[i,"Time"]},ignore_index=True)

In [9]:
for i in range(23):
    po=po.append({'current':df.at[i,"Current Ideal"],'load':df.at[i,"P_L Ideal"],'result':0,'Time':df.at[i,"Time"]},ignore_index=True)

In [10]:
po = pd.concat([po,pd.get_dummies(po['Time'], prefix='Time',dummy_na=True)],axis=1).drop(['Time'],axis=1)

In [11]:
po

,current,load,result,Time_1.0,Time_2.0,Time_3.0,Time_4.0,Time_5.0,Time_6.0,Time_7.0,...,Time_15.0,Time_16.0,Time_17.0,Time_18.0,Time_19.0,Time_20.0,Time_21.0,Time_22.0,Time_23.0,Time_nan
0,8.858000e+00,492.0,1.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4.429000e+00,246.0,1.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4.429000e+00,246.0,1.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4.429000e+00,246.0,1.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4.429000e+00,246.0,1.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,7.382000e+00,410.0,1.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,1.476000e+01,819.9,1.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,2.657000e+01,1476.0,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2.953000e+01,1640.0,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1.476000e+01,819.9,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
poo=po.values

In [13]:
yy=poo[:,2]
po.drop(['result'],axis="columns",inplace=True)

In [14]:
XX=poo[:,:]

In [15]:
def neural_net():
    model = Sequential()
    model.add(Dense(16, input_dim=27, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, kernel_initializer='normal',activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [16]:
from keras.utils import np_utils

In [17]:
encoder = LabelEncoder()
encoder.fit(yy)
encoded_Y = encoder.transform(yy)
dummy_y = np_utils.to_categorical(encoded_Y)

In [18]:
dummy_y

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],


In [19]:
dummy_XX=XX

In [20]:
scaler=StandardScaler()

In [21]:
dummy_XX=scaler.fit_transform(dummy_XX)

In [22]:
dummy_XX

array([[-0.13554233, -0.94174016, -0.4472136 , ..., -0.21320072,
        -0.21320072,  0.        ],
       [-0.53729721, -1.06302048, -0.4472136 , ..., -0.21320072,
        -0.21320072,  0.        ],
       [-0.53729721, -1.06302048, -0.4472136 , ..., -0.21320072,
        -0.21320072,  0.        ],
       ...,
       [-0.93905208,  2.37276216, -1.34164079, ..., -0.21320072,
        -0.21320072,  0.        ],
       [-0.93905454,  2.53446925, -1.34164079, ...,  4.69041576,
        -0.21320072,  0.        ],
       [-0.93903975,  1.7259338 , -1.34164079, ..., -0.21320072,
         4.69041576,  0.        ]])

In [24]:
mm=neural_net()
history=mm.fit(XX,dummy_y,epochs=1000)

Epoch 1/1000
92/92 [==============================] - 0s 2ms/step - loss: 1.6567 - accuracy: 0.2826
Epoch 2/1000
92/92 [==============================] - 0s 86us/step - loss: 1.5129 - accuracy: 0.3261
Epoch 3/1000
92/92 [==============================] - 0s 116us/step - loss: 1.4072 - accuracy: 0.1957
Epoch 4/1000
92/92 [==============================] - 0s 70us/step - loss: 1.4356 - accuracy: 0.2283
Epoch 5/1000
92/92 [==============================] - 0s 67us/step - loss: 1.3984 - accuracy: 0.2174
Epoch 6/1000
92/92 [==============================] - 0s 88us/step - loss: 1.4106 - accuracy: 0.1957
Epoch 7/1000
92/92 [==============================] - 0s 81us/step - loss: 1.4136 - accuracy: 0.2609
Epoch 8/1000
92/92 [==============================] - 0s 78us/step - loss: 1.3904 - accuracy: 0.3370
Epoch 9/1000
92/92 [==============================] - 0s 76us/step - loss: 1.3840 - accuracy: 0.2391
Epoch 10/1000
92/92 [==============================] - 0s 94us/step - loss: 1.3682 - accura